In [1]:
%matplotlib inline
import numpy as np
import struct as st
import matplotlib.pyplot as plt

In [2]:
file_test_x = 't10k-images.idx3-ubyte'
file_test_y = 't10k-labels.idx1-ubyte'
file_train_x = 'train-images.idx3-ubyte'
file_train_y = 'train-labels.idx1-ubyte'

In [3]:
def read_labels(labels_idx_file):
    file = open(labels_idx_file, 'rb')
    file.seek(0)
    st.unpack('>4B', file.read(4))
    NUM_SAMPLES = st.unpack('>I', file.read(4))[0]
    outcome_array = np.zeros((NUM_SAMPLES,1), dtype = np.int32)
    for i in range(NUM_SAMPLES):
        result = st.unpack('>1B', file.read(1))
        outcome_array[i] = np.int32(result)
    return outcome_array

def read_data(data_idx_file):
    file = open(data_idx_file, 'rb')
    file.seek(0)
    st.unpack('>4B', file.read(4))
    NUM_SAMPLES = st.unpack('>I', file.read(4))[0]
    IMG_WIDTH = st.unpack('>I', file.read(4))[0]
    IMG_HEIGHT = st.unpack('>I', file.read(4))[0]
    outcome_array = np.zeros((NUM_SAMPLES, IMG_WIDTH, IMG_HEIGHT))
    for i in range(NUM_SAMPLES):
        result = st.unpack('>'+str(IMG_WIDTH*IMG_HEIGHT)+'B',file.read(IMG_WIDTH*IMG_HEIGHT))
        result = np.asarray(result).reshape((IMG_WIDTH,IMG_HEIGHT))
        outcome_array[i] = result
    return outcome_array

In [4]:
x_test_proba = read_data(file_test_x)
x_train_proba = read_data(file_train_x)
y_test_proba = read_labels(file_test_y)
y_train_proba = read_labels(file_train_y)

In [5]:
x_train, y_train = x_train_proba.reshape(x_train_proba.shape[0], -1), y_train_proba
x_test, y_test = x_test_proba.reshape(x_test_proba.shape[0], -1), y_test_proba

In [6]:
shuffle_index = np.random.permutation(60000)
x_train, y_train = x_train[shuffle_index], y_train[shuffle_index]

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()

In [8]:
from sklearn.model_selection import RandomizedSearchCV

In [9]:
param_dist = {'n_neighbors': [2, 5, 7], 
              'weights': ['uniform', 'distance']
              }
random_search = RandomizedSearchCV (knn_clf, param_distributions=param_dist, n_iter = 6, 
                                    scoring = "accuracy", n_jobs=-1,cv=3, random_state=42)
random_search.fit(x_train, y_train)

C:\Users\Timur\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params=None, iid='warn', n_iter=6, n_jobs=-1,
          param_distributions={'n_neighbors': [2, 5, 7], 'weights': ['uniform', 'distance']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [10]:
random_search.best_params_

{'weights': 'distance', 'n_neighbors': 2}

In [15]:
random_search.score

<bound method BaseSearchCV.score of RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params=None, iid='warn', n_iter=6, n_jobs=-1,
          param_distributions={'n_neighbors': [2, 5, 7], 'weights': ['uniform', 'distance']},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)>

In [17]:
random_search.best_score_

0.9705666666666667